# Passo 1: Monta Google Drive

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

# NOTA: Dopo aver eseguito questa cella, dovrai cliccare su un link
# autorizzare il tuo account Google e incollare il codice di autorizzazione.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Passo 2: Naviga nella Cartella del Progetto

In [ ]:
import os
# ATTENZIONE: Modifica questo percorso perché corrisponda a dove hai salvato
# la cartella del tuo progetto su Google Drive.
PROJECT_PATH = '/content/drive/MyDrive/basketball_analysis/basketball_analysis'

os.chdir(PROJECT_PATH)

# Verifica di essere nella cartella giusta listando i file
!ls

ball_acquisition		shot_detector
basketball-court-detection-2-1	shot_visualizer
configs				speed_and_distance_calculator
court_keypoint_detector		stubs
Dockerfile			tactical_view
drawers				tactical_view_converter
extract_frames.py		team_assigner
GEMINI.md			test_ball_drawer_output.jpg
generate_stubs.py		test_ball_drawer.py
images				test_court_drawer_output.jpg
input_videos			test_court_drawer.py
main.py				test_player_drawer_output.jpg
models				test_player_drawer.py
output_videos			test_shot_visualizer_output.jpg
pass_and_interception_detector	test_shot_visualizer.py
README.md			trackers
requirements.txt		training_notebooks
roboflow_dataset		utils
runs				yolo11n.pt
shot_classifier


Dovresti vedere l'elenco dei file del tuo progetto (`main.py`, `models/`, `input_videos/`, ecc.).


# Passo 3: Installa le Librerie Necessarie

In [ ]:
# Installa le versioni più recenti di Ultralytics e Roboflow
!pip install ultralytics roboflow -q

L'opzione `-q` (quiet) riduce l'output, mantenendo il notebook più pulito.

# Cella 4: Download del Dataset

In [ ]:
from roboflow import Roboflow

# Incolla qui lo snippet di codice che hai copiato da Roboflow.
# Dovrebbe essere simile a questo, ma con i tuoi dati specifici.
# ASSICURATI DI USARE LO SNIPPET DEL NUOVO DATASET (quello che abbiamo appena creato)

rf = Roboflow(api_key="4tJFP3EhC2GQwZbgktvs")
project = rf.workspace("firstworkspace-riuxx").project("basketball-court-detection-2-ny1gt")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


Importante: Assicurati di usare lo snippet corretto del progetto "basketball-court-detection-2" e della versione che hai appena generato.


# Passo 5: Esegui il Fine-Tuning (La Cella Cruciale)

Questa è la cella che sostituisce il tuo vecchio comando `!yolo`. Useremo le API Python.

In [ ]:
from ultralytics import YOLO

# 1. Carica il TUO modello pre-addestrato, non uno generico.
#   Questo è il cuore del fine-tuning.
model = YOLO('models/court_keypoint_detector_model_v2.pt')

# 2. Avvia il processo di training usando il dataset scaricato.
results = model.train(
    data=f'{dataset.location}/data.yaml',  # Percorso al file di configurazione del dataset
   epochs=100,                             # Numero di cicli di addestramento. 100 è un buon inizio.
   imgsz=640,                              # Dimensione delle immagini, deve corrispondere a quella di Roboflow.
   project='runs/pose',                    # Cartella dove salvare i risultati
   name='finetune_court_detector'          # Nome specifico per questo esperimento
)

FileNotFoundError: [Errno 2] No such file or directory: 'models/court_keypoint_detector_model_v2.pt'

Spiegazione delle differenze chiave:

   * `model = YOLO(...)`: Invece di specificare il modello nella riga di comando, lo carichiamo prima come oggetto Python. Questo è il punto in cui carichiamo
     court_keypoint_detector_model_v2.pt.
   * `model.train(...)`: Chiamiamo il metodo .train() sull'oggetto model. Questo ci dà un controllo molto più pulito sui parametri.
   * `data=f'{dataset.location}/data.yaml'`: Usiamo una f-string per passare dinamicamente la posizione del file data.yaml che Roboflow ci ha fornito. È più robusto che
     scriverlo a mano.
   * `epochs=100`: Ho impostato 100 epoche. È un numero solido per il fine-tuning. Il tuo precedente epochs=500 era probabilmente eccessivo e avrebbe rischiato l'overfitting.
     YOLO ha un meccanismo di "early stopping", quindi si fermerà da solo se non vede miglioramenti per un certo numero di epoche (di default 50).
   * `project` e `name`: Questi parametri ci permettono di organizzare meglio i risultati. Il tuo nuovo modello e i grafici di training verranno salvati in
     runs/pose/finetune_court_detector/.

In [ ]:
from google.colab import files
files.download('runs/pose/finetune_court_detector/weights/best.pt')
# files.download('runs/pose/finetune_court_detector/weights/last.pt')

# Passo 6: Verifica i Risultati

Dopo che la Cella 5 ha finito, il tuo nuovo modello sarà stato salvato. Puoi verificare il percorso esatto e usarlo. Il file che ti interessa sarà in:
  `runs/pose/finetune_court_detector/weights/best.pt`

Questo è il file che dovrai scaricare da Google Drive e usare nella tua applicazione principale.

Questo nuovo notebook è più pulito, più robusto e, soprattutto, esegue correttamente il fine-tuning come avevamo pianificato.